In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from datetime import timedelta

# visualization
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns

%matplotlib inline

# configure visualizations
sns.set_style('whitegrid')
figsize=(8,6)

In [2]:
def load_df(path, columns=None, use_threads=True):
    try:
        return pq.read_table(path, columns=columns, use_threads=use_threads).to_pandas()
    except Exception as e:
        print(e)

def downcast_ints(df):
    # downcast int types
    df_int = df.select_dtypes(include=['int'])
    converted_int = df_int.apply(pd.to_numeric, downcast='unsigned')
    df[converted_int.columns] = converted_int
    
    return df
        
def prepare(path,cols):

    df = load_df(path,cols)
    df = df.reset_index(drop=True)
    df = downcast_ints(df)
    
    return df

In [3]:
def pieOthers(dfVars,dfVals,threshold):
    df = pd.DataFrame(
        data = {'var': dfVars, 'value' : dfVals},
        ).sort_values('value', ascending = False)
    
    if threshold > 0 :
        df2 = df[:threshold].copy()
        new_row = pd.DataFrame(data = {
            'var' : ['Others'],
            'value' : [dfVals[threshold:].sum()]
        })

        df2 = pd.concat([df2, new_row])
    else:
        df2 = df
        
    percent = 100.*df2['value']/df2['value'].sum()
    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(df2['var'], percent)]
    #labels = ['{0} - {1}'.format(i,j) for i,j in zip(df2['var'], df2['value'])]
    
    colorsX = plt.cm.tab20.colors
    patches, texts = plt.pie(df2['value'], shadow=True, startangle=90, colors = colorsX)
    plt.legend(patches, labels, loc="best")
    plt.axis('equal')
    plt.tight_layout()
    plt.show()
    
def pieOthers2(dfVars,dfVals,threshold):
    df = pd.DataFrame(
        data = {'var': dfVars, 'value' : dfVals},
        ).sort_values('value', ascending = False)
    
    if threshold > 0 :
        df2 = df[:threshold].copy()
        new_row = pd.DataFrame(data = {
            'var' : ['Others'],
            'value' : [dfVals[threshold:].sum()]
        })

        df2 = pd.concat([df2, new_row])
    else:
        df2 = df
    
    percent = 100.*df2['value']/df2['value'].sum()
    labels = ['{0} - {1:1.2f} %'.format(i,j) for i,j in zip(df2['var'], percent)]
    
    #labels = ['{0} - {1}'.format(i,j) for i,j in zip(df2['var'], df2['value'])]
    
    fig=plt.figure(figsize=(7,5))
    gs1 = gridspec.GridSpec(1,1,
        left=0.1,right=0.7,
        bottom=0.1,top=0.7,
    )
    pie_ax=fig.add_subplot(gs1[0])
    
    colors = plt.cm.tab20.colors
    wedges, texts = pie_ax.pie(
        df2['value'],
        shadow=True,
        colors=colors,
        startangle=90,
    )
    bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
    kw = dict(xycoords='data', textcoords='data', arrowprops=dict(arrowstyle="-"), zorder=0, va="center")

    for i, p in enumerate(wedges):
        ang = (p.theta2 - p.theta1)/2. + p.theta1
        y = np.sin(np.deg2rad(ang))
        x = np.cos(np.deg2rad(ang))
        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = "angle,angleA=0,angleB={}".format(ang)
        kw["arrowprops"].update({"connectionstyle": connectionstyle,"color":colors[i]})
        pie_ax.annotate(labels[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                     horizontalalignment=horizontalalignment, **kw)
    pie_ax.axis('equal')

    plt.show()

In [4]:
cols = ['id','network_status', 'screen_on', 'timezone', 'country_code', 'period', 'direction', 'ppm']
dfSamples = prepare('datasets/samplesPPM.parquet',cols)

In [9]:
dfSamples_droppedDup = dfSamples.drop(['id', 'screen_on', 'network_status'], axis=1).drop_duplicates()

#due to functional dependicies, individual duplication drops must be done
dfSamples_droppedDup_Screen = dfSamples.drop(dfSamples.columns.difference(['period', 'direction', 'ppm', 'screen_on']), axis=1).drop_duplicates()
dfSamples_droppedDup_NetworkStatus = dfSamples.drop(dfSamples.columns.difference(['period', 'direction', 'ppm', 'network_status']), axis=1).drop_duplicates()


In [43]:
def samplesGroupBy(dataframe, var, direction, minCount):
    df = dataframe.groupby([var,'direction'])['ppm'].agg(average_ppm='mean', std='std', count='count').reset_index().sort_values(['average_ppm'], ascending=False)
    return df.loc[(df['direction'] == direction) & (df['count'] >= minCount)]

dfTimezoneGBPos = samplesGroupBy(dfSamples_droppedDup, 'timezone', 1, 30)
dfTimezoneGBNeg = samplesGroupBy(dfSamples_droppedDup, 'timezone', -1, 30)

dfCountryGBPos = samplesGroupBy(dfSamples_droppedDup, 'country_code', 1, 30)
dfCountryGBNeg = samplesGroupBy(dfSamples_droppedDup, 'country_code', -1, 30)

dfScreenGBPos = samplesGroupBy(dfSamples_droppedDup_Screen, 'screen_on', 1, 0)
dfScreenGBNeg = samplesGroupBy(dfSamples_droppedDup_Screen, 'screen_on', -1, 0)

dfNetworkStatusGBPos = samplesGroupBy(dfSamples_droppedDup_NetworkStatus, 'network_status', 1, 30)
dfNetworkStatusGBNeg = samplesGroupBy(dfSamples_droppedDup_NetworkStatus, 'network_status', -1, 30)

In [16]:
print(dfTimezoneGBPos);print(dfTimezoneGBNeg)

               timezone  direction  average_ppm       std  count
61    AFRICA/PORTO-NOVO        1.0     2.399762  2.791198     32
249       ASIA/JAYAPURA        1.0     1.033726  0.488529    472
370     EUROPE/ISTANBUL        1.0     1.018886  0.705314    155
382   EUROPE/LUXEMBOURG        1.0     0.911628  0.495417    513
304        ASIA/TBILISI        1.0     0.907625  0.405421    104
..                  ...        ...          ...       ...    ...
310          ASIA/TOKYO        1.0     0.399742  0.149462     86
318  ASIA/YEKATERINBURG        1.0     0.370269  0.225378     91
223         ASIA/BEIRUT        1.0     0.334566  0.108292     44
57       AFRICA/NAIROBI        1.0     0.309540  0.169422     80
202   AMERICA/VANCOUVER        1.0     0.299367  0.127781     82

[137 rows x 5 columns]
              timezone  direction  average_ppm       std  count
15    AFRICA/BUJUMBURA       -1.0     2.644638  2.297229     60
60   AFRICA/PORTO-NOVO       -1.0     2.112202  2.329160     46
112 

In [17]:
print(dfCountryGBPos);print(dfCountryGBNeg)

    country_code  direction  average_ppm       std  count
27            bj        1.0     2.092857  2.714462     35
266           tr        1.0     1.003432  0.566540    148
163           lu        1.0     0.918329  0.491773    500
1             ae        1.0     0.893526  0.613443    411
274           ua        1.0     0.891641  0.801452    409
..           ...        ...          ...       ...    ...
39            bw        1.0     0.417514  0.101347    118
81            eg        1.0     0.410910  0.208281     59
270           tw        1.0     0.403103  0.281912     38
143           jp        1.0     0.401657  0.164492     49
145           ke        1.0     0.323094  0.172040     72

[94 rows x 5 columns]
    country_code  direction  average_ppm       std  count
24            bi       -1.0     2.638927  2.280685     58
26            bj       -1.0     1.898302  2.258115     48
292           zw       -1.0     0.882162  0.714865     94
34            bs       -1.0     0.784886  0.64882

In [21]:
print(dfScreenGBPos);print(dfScreenGBNeg)

   screen_on  direction  average_ppm       std   count
1          0        1.0     0.682100  0.679152  128211
3          1        1.0     0.647396  0.693152   87967
   screen_on  direction  average_ppm       std   count
2          1       -1.0     0.324557  0.922686  131032
0          0       -1.0     0.229028  0.495743  102907


In [44]:
print(dfNetworkStatusGBPos);print(dfNetworkStatusGBNeg)

   network_status  direction  average_ppm       std  count
23         EVDO_A        1.0     0.763882  0.275708    140
31          HSPAP        1.0     0.678161  0.495035   9522
39           WIFI        1.0     0.671215  0.830926  82410
29           HSPA        1.0     0.665793  0.625496   3692
15   DISCONNECTED        1.0     0.662881  0.601121  56928
37           UTMS        1.0     0.662144  0.455777   3871
27          HSDPA        1.0     0.632587  0.310541    562
35            LTE        1.0     0.627449  0.467582  19282
1               0        1.0     0.625242  0.405350   1632
19           EDGE        1.0     0.591348  0.361806   1560
33          HSUPA        1.0     0.578681  0.298708    688
17  DISCONNECTING        1.0     0.538634  0.283396    947
25           GPRS        1.0     0.522476  0.308337    129
6              19        1.0     0.465028  0.222995     47
   network_status  direction  average_ppm       std  count
28           HSPA       -1.0     0.343693  0.500024  100

In [48]:
cols2 = ['id','sample_id','health']
dfBatteryDetails = prepare('datasets/battery_details.parquet',cols2)

#only consider samples with battery health = {OVERHEAT, OVER VOLTAGE, GOOD}. Leave out = {DEAD, UNSPECIFIED FAILURE, UNKNOWN}
#dfBatteryDetails_Health = dfBatteryDetails.loc[(dfBatteryDetails['health'] == "GOOD") | (dfBatteryDetails['health'] == 'OVER VOLTAGE') | (dfBatteryDetails['health'] == "OVERHEAT")]
dfBatteryDetails_Health = dfBatteryDetails.loc[(dfBatteryDetails['health'] == "GOOD")]

In [49]:
df_inner_SamplesBattery = pd.merge(dfSamples, dfBatteryDetails_Health, left_on='id', right_on='sample_id', how='inner')

In [50]:
df_inner_SamplesBattery_droppedDup = df_inner_SamplesBattery.drop(['id_x', 'screen_on', 'network_status', 'id_y', 'sample_id', 'health'], axis=1).drop_duplicates()
df_inner_SamplesBattery_droppedDup_Screen = df_inner_SamplesBattery.drop(df_inner_SamplesBattery.columns.difference(['period', 'direction', 'ppm', 'screen_on']), axis=1).drop_duplicates()
df_inner_SamplesBattery_droppedDup_NetworkStatus = df_inner_SamplesBattery.drop(df_inner_SamplesBattery.columns.difference(['period', 'direction', 'ppm', 'network_status']), axis=1).drop_duplicates()

In [51]:
df_inner_SamplesBattery_TimezoneGBPos = samplesGroupBy(df_inner_SamplesBattery_droppedDup, 'timezone', 1, 30)
df_inner_SamplesBattery_TimezoneGBNeg = samplesGroupBy(df_inner_SamplesBattery_droppedDup, 'timezone', -1, 30)

df_inner_SamplesBattery_CountryGBPos = samplesGroupBy(df_inner_SamplesBattery_droppedDup, 'country_code', 1, 30)
df_inner_SamplesBattery_CountryGBNeg = samplesGroupBy(df_inner_SamplesBattery_droppedDup, 'country_code', -1, 30)

df_inner_SamplesBattery_ScreenGBPos = samplesGroupBy(df_inner_SamplesBattery_droppedDup_Screen, 'screen_on', 1, 0)
df_inner_SamplesBattery_ScreenGBNeg = samplesGroupBy(df_inner_SamplesBattery_droppedDup_Screen, 'screen_on', -1, 0)

df_inner_SamplesBattery_NetworkStatusGBPos = samplesGroupBy(df_inner_SamplesBattery_droppedDup_NetworkStatus, 'network_status', 1, 30)
df_inner_SamplesBattery_NetworkStatusGBNeg = samplesGroupBy(df_inner_SamplesBattery_droppedDup_NetworkStatus, 'network_status', -1, 30)

In [52]:
print(dfTimezoneGBPos);print(df_inner_SamplesBattery_TimezoneGBPos)
print(dfTimezoneGBNeg);print(df_inner_SamplesBattery_TimezoneGBNeg)

               timezone  direction  average_ppm       std  count
61    AFRICA/PORTO-NOVO        1.0     2.399762  2.791198     32
249       ASIA/JAYAPURA        1.0     1.033726  0.488529    472
370     EUROPE/ISTANBUL        1.0     1.018886  0.705314    155
382   EUROPE/LUXEMBOURG        1.0     0.911628  0.495417    513
304        ASIA/TBILISI        1.0     0.907625  0.405421    104
..                  ...        ...          ...       ...    ...
310          ASIA/TOKYO        1.0     0.399742  0.149462     86
318  ASIA/YEKATERINBURG        1.0     0.370269  0.225378     91
223         ASIA/BEIRUT        1.0     0.334566  0.108292     44
57       AFRICA/NAIROBI        1.0     0.309540  0.169422     80
202   AMERICA/VANCOUVER        1.0     0.299367  0.127781     82

[137 rows x 5 columns]
               timezone  direction  average_ppm       std  count
61    AFRICA/PORTO-NOVO        1.0     2.399762  2.791198     32
249       ASIA/JAYAPURA        1.0     1.033726  0.488529    472
3

In [53]:
print(dfCountryGBPos);print(df_inner_SamplesBattery_CountryGBPos)
print(dfCountryGBNeg);print(df_inner_SamplesBattery_CountryGBNeg)

    country_code  direction  average_ppm       std  count
27            bj        1.0     2.092857  2.714462     35
266           tr        1.0     1.003432  0.566540    148
163           lu        1.0     0.918329  0.491773    500
1             ae        1.0     0.893526  0.613443    411
274           ua        1.0     0.891641  0.801452    409
..           ...        ...          ...       ...    ...
39            bw        1.0     0.417514  0.101347    118
81            eg        1.0     0.410910  0.208281     59
270           tw        1.0     0.403103  0.281912     38
143           jp        1.0     0.401657  0.164492     49
145           ke        1.0     0.323094  0.172040     72

[94 rows x 5 columns]
    country_code  direction  average_ppm       std  count
27            bj        1.0     2.092857  2.714462     35
266           tr        1.0     1.003432  0.566540    148
163           lu        1.0     0.918329  0.491773    500
1             ae        1.0     0.893526  0.61344

In [54]:
print(dfScreenGBPos);print(df_inner_SamplesBattery_ScreenGBPos)
print(dfScreenGBNeg);print(df_inner_SamplesBattery_ScreenGBNeg)

   screen_on  direction  average_ppm       std   count
1          0        1.0     0.682100  0.679152  128211
3          1        1.0     0.647396  0.693152   87967
   screen_on  direction  average_ppm       std   count
1          0        1.0     0.682205  0.679186  128141
3          1        1.0     0.646629  0.690513   87751
   screen_on  direction  average_ppm       std   count
2          1       -1.0     0.324557  0.922686  131032
0          0       -1.0     0.229028  0.495743  102907
   screen_on  direction  average_ppm       std   count
2          1       -1.0     0.323293  0.916810  130647
0          0       -1.0     0.228589  0.495694  102618


In [55]:
print(dfNetworkStatusGBPos);print(df_inner_SamplesBattery_NetworkStatusGBPos)
print(dfNetworkStatusGBNeg);print(df_inner_SamplesBattery_NetworkStatusGBNeg)

   network_status  direction  average_ppm       std  count
23         EVDO_A        1.0     0.763882  0.275708    140
31          HSPAP        1.0     0.678161  0.495035   9522
39           WIFI        1.0     0.671215  0.830926  82410
29           HSPA        1.0     0.665793  0.625496   3692
15   DISCONNECTED        1.0     0.662881  0.601121  56928
37           UTMS        1.0     0.662144  0.455777   3871
27          HSDPA        1.0     0.632587  0.310541    562
35            LTE        1.0     0.627449  0.467582  19282
1               0        1.0     0.625242  0.405350   1632
19           EDGE        1.0     0.591348  0.361806   1560
33          HSUPA        1.0     0.578681  0.298708    688
17  DISCONNECTING        1.0     0.538634  0.283396    947
25           GPRS        1.0     0.522476  0.308337    129
6              19        1.0     0.465028  0.222995     47
   network_status  direction  average_ppm       std  count
23         EVDO_A        1.0     0.763882  0.275708    1